## III - Неопределенность оптимального портфеля. Оптимизация CVaR. 

Целью работы является оценка неопределенности оптимального портфеля для нормального многомерного распределения и двух способов вычисления оптимального портфеля (оптимизация в модели Марковица и оптимизация CVaR)

### Подготовка модели

#### Загружаем тикеры с фоднового рынка NASDAQ

In [430]:
import pandas as pd

DATA_PATH    = './downloader-data'
TICKERS_PATH = './tickers'

def read_tickers(stock_markets) -> pd.DataFrame:
    '''Returns pandas dataframe containing all tickers for the @stock_markets '''
    ticker_files = [f'{TICKERS_PATH}/{sm}.csv' for sm in stock_markets]
    tickers = pd.concat([pd.read_csv(tf) for tf in ticker_files], ignore_index=True)
    return tickers


stock_markets = ['NASDAQ']
tickers = read_tickers(stock_markets)
tickers

,ticker,company
0,AAIT,iShares MSCI All Country Asia Information Tech...
1,AAL,"American Airlines Group, Inc."
2,AAME,Atlantic American Corporation
3,AAOI,"Applied Optoelectronics, Inc."
4,AAON,"AAON, Inc."
...,...,...
2962,ZN,Zion Oil & Gas Inc
2963,ZNGA,Zynga Inc.
2964,ZSPH,"ZS Pharma, Inc."
2965,ZU,"zulily, inc."


#### Загружаем исторические данные за 2021 год для полученных тикеров

In [431]:
def read_historical_data(tickers):
    '''Returns dict {ticker : historical data for the ticker}'''
    data_for_ticker = {}
    for index, (ticker, name) in tickers.iterrows():
        try:
            data = pd.read_csv(f'{DATA_PATH}/{ticker}.csv')
            if len(data) > 100:
                data_for_ticker[ticker] = data
        except:
            pass
    return data_for_ticker


data_for_ticker = read_historical_data(tickers)

#### Пример данных для AAPL:

In [432]:
data_for_ticker['AAPL']

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2020-11-30,116.597423,120.584682,116.437929,118.670799,169410200,0.0,0
1,2020-12-01,120.624557,123.076720,119.627742,122.329109,127728200,0.0,0
2,2020-12-02,121.631329,122.977035,120.504931,122.687958,89004200,0.0,0
3,2020-12-03,123.126555,123.385729,121.820730,122.548409,78967600,0.0,0
4,2020-12-04,122.209495,122.468669,121.132933,121.860611,78260400,0.0,0
...,...,...,...,...,...,...,...,...
248,2021-11-23,161.119995,161.800003,159.059998,161.410004,96041900,0.0,0
249,2021-11-24,160.750000,162.139999,159.639999,161.940002,69463600,0.0,0
250,2021-11-26,159.570007,160.449997,156.360001,156.809998,76959800,0.0,0
251,2021-11-29,159.369995,161.190002,158.789993,160.240005,88748200,0.0,0


#### Выбираем 20 активов

In [433]:
assets = ['MDLZ', 'MSFT', 'NXPI', 'PCAR', 'INTC',
          'NVDA', 'ILMN', 'DXCM', 'ROST', 'LULU']

assets_data = {}
for asset in assets:
    assets_data[asset] = data_for_ticker[asset]

#### Оцениваем математические ожидания доходностей

In [434]:
import pandas as pd
import numpy as np

def add_logret(ticker_data: pd.DataFrame, by_column='Close') -> pd.DataFrame:
    '''Returns @ticker_data with calculated "logret" and "-logret" columns'''
    ticker_data = ticker_data.assign(logret=np.log(ticker_data[by_column]).diff())
    ticker_data['-logret'] = ticker_data['logret'].mul(-1)
    return ticker_data


def get_logret_mean_std(tickers, data_map, by_column='Close') -> pd.DataFrame:
    '''Returns @result pd.DataFrame such that @result.loc[ticker] == [logret_mean, logret_std]'''
    result = pd.DataFrame(data=[], columns=['ticker', 'logret_mean', 'logret_std'])
    result.set_index('ticker', inplace=True)

    for ticker in tickers:
        ticker_data = data_map[ticker]
        ticker_logret = np.log(ticker_data[by_column]).diff()
        result.loc[ticker] = [ticker_logret.mean(), ticker_logret.std()]
    
    return result
  

for ticker in assets_data.keys():
    assets_data[ticker] = add_logret(assets_data[ticker])

estims = get_logret_mean_std(assets, assets_data)
estims

,logret_mean,logret_std
ticker,,
MDLZ,0.000164,0.009635
MSFT,0.001743,0.012638
NXPI,0.001406,0.023336
PCAR,-0.000078,0.015266
INTC,0.000118,0.020613
NVDA,0.003541,0.025897
ILMN,0.000500,0.020907
DXCM,0.002243,0.022329
ROST,0.000076,0.018766


#### Находим матрицу выборочных ковариаций доходностей 

In [435]:
def get_covariation_matrix(tickers,                          
                           data_map=data_for_ticker,
                           by_column='logret'):
    columns = pd.DataFrame()
    for ticker in tickers:
        columns[ticker] = data_map[ticker][by_column][1:]
        
    matrix = columns.cov()
    return matrix

cov_matrix = get_covariation_matrix(assets, data_map=assets_data)
cov_matrix

,MDLZ,MSFT,NXPI,PCAR,INTC,NVDA,ILMN,DXCM,ROST,LULU
MDLZ,0.000093,0.000032,0.000031,0.000024,0.000026,0.000018,0.000022,0.000026,0.000058,0.000038
MSFT,0.000032,0.000160,0.000129,0.000025,0.000089,0.000184,0.000097,0.000112,0.000056,0.000091
NXPI,0.000031,0.000129,0.000545,0.000112,0.000250,0.000343,0.000170,0.000184,0.000152,0.000192
PCAR,0.000024,0.000025,0.000112,0.000233,0.000090,0.000058,0.000025,0.000008,0.000089,0.000033
INTC,0.000026,0.000089,0.000250,0.000090,0.000425,0.000188,0.000103,0.000102,0.000111,0.000103
NVDA,0.000018,0.000184,0.000343,0.000058,0.000188,0.000671,0.000160,0.000203,0.000075,0.000210
ILMN,0.000022,0.000097,0.000170,0.000025,0.000103,0.000160,0.000437,0.000202,0.000012,0.000133
DXCM,0.000026,0.000112,0.000184,0.000008,0.000102,0.000203,0.000202,0.000499,0.000068,0.000161
ROST,0.000058,0.000056,0.000152,0.000089,0.000111,0.000075,0.000012,0.000068,0.000352,0.000105
LULU,0.000038,0.000091,0.000192,0.000033,0.000103,0.000210,0.000133,0.000161,0.000105,0.000378


In [436]:
import plotly.express as px

fig = px.imshow(cov_matrix, title='Covariation Matrix',
                color_continuous_scale=px.colors.diverging.RdYlGn[::-1])
fig.show()

#### Проверяем вырожденность матрицы и число обусловленности

In [437]:
det = np.linalg.det(cov_matrix)
print(f'det = {det}')

con = np.linalg.cond(cov_matrix, p='fro')
print(f'con = {con}')

det = 1.1829872583873188e-36
con = 37.80303663511966


### 1. Истинный оптимальный портфель в модели Марковица с заданным отношением к риску. 

С заданным отношением к риску  подобираем константу b таким образом, что истинный оптимальный CVaR портфель совпадает с истинным оптимальным портфелем п.1. Значение константы смотри в упражнениях к теме.

$$ b = \frac{1}{\sqrt{2 \pi }} \frac{1}{(1 - \beta)} exp(-(\Phi^{-1}(\beta))^2 / 2)$$

In [438]:
import math
from scipy.stats import norm

beta = 0.90
b = (1 / math.sqrt(2 * math.pi)) * (1 / (1 - beta)) * np.exp(-(norm.ppf(beta)**2 / 2)) 
print(f'b = {b}')

b = 1.754983319324869


#### Решаем задачу оптимизации 

$$ -E(x)+ b \cdot σ(x) \rightarrow min $$
при условиях:
$$ x_1 + x_2 + ... +x_N = 1 $$
$$ x_i >= 0 $$
где
$$ E(x)= E_1x_1 + E_2 x_2 + ... + E_Nx_N $$
$$ σ^2(x)=\Sigma\Sigma σ_{i,j} x_i \cdot x_j $$

In [439]:
import math 
import numpy as np
from scipy.optimize import minimize

def get_E(x, vecE=estims):
    E_i = _estims.loc[ticker][0]
    E = sum([(E_i * x_i) for x_i in x]) 
    return E


def get_sigma(x, _cov_matrix=cov_matrix):
    sigma_squared = 0
    
    for i in range(len(x)):
        for j in range(len(x)):
            simga_i_j = cov_matrix.iloc[i].iloc[j]
            sigma_squared += simga_i_j * x[i] * x[j]
    
    sigma = math.sqrt(sigma_squared)
    return sigma
            
            
def target_function(x):
    E = get_E(x)
    sigma = get_sigma(x)
    result = -E + b * sigma
    return result


def find_optimal():
    x0 = np.array([1/len(assets)] * len(assets))
    solution = minimize(get_F, x0, method='SLSQP', 
                        constraints=[{'type': 'eq',  'fun': lambda x: sum(x) - 1}],
                        bounds=[(0, 1)] * len(assets))
    if not solution.success:
        raise Exception(opt.message)
    return solution
    
opt_solution = find_optimal()

#### Веса портфеля:

In [440]:
import plotly.express as px

print(f'Сумма весов: {sum(opt_solution.x)}')
fig = px.bar(x=assets, y=opt_solution.x)
fig.show()

Сумма весов: 1.0000000000000002


#### Значение целевой функции:

In [441]:
print(opt_solution.fun)

0.013025749039869479


### 2. Оценка неопределенности оптимального портфеля в модели Марковица с заданным отн. к риску. 

#### 2.1 Задаём число наблюдений T=30. С помощью генератора многомерного нормального распределения создаём выборку размера Т из нормального распределения с вектором математических ожиданий  E=(E1, E2, …, EN) и матрицей ковариаций (σi,j). 

In [442]:
T = 30
sample_raw = np.random.multivariate_normal(estims['logret_mean'], cov_matrix, T)
sample = pd.DataFrame(columns=assets, data=sample_raw)
sample

,MDLZ,MSFT,NXPI,PCAR,INTC,NVDA,ILMN,DXCM,ROST,LULU
0,0.007479,-0.039593,-0.005117,0.000949,0.004005,-0.036783,0.000142,0.004998,0.001106,-0.009431
1,0.002501,-0.005137,0.026244,0.011273,-0.003912,0.023029,-0.002404,0.003539,-0.009493,0.010947
2,0.004211,0.012943,0.027686,0.018151,-0.009349,0.054353,-0.006242,-0.009778,-0.020432,0.029544
3,-0.005456,-0.012730,-0.000484,-0.001018,-0.013396,-0.024460,-0.014248,-0.023700,-0.001717,-0.016598
4,0.015682,-0.019194,-0.032180,-0.002569,-0.009944,-0.046742,-0.013381,-0.029239,0.010533,0.007414
5,-0.001623,-0.011828,0.006227,0.002793,0.003207,0.006315,-0.014719,-0.009278,0.024688,-0.017685
6,-0.000889,-0.005983,-0.032813,-0.003649,-0.014388,0.004120,0.007821,-0.025345,0.012637,-0.000574
7,-0.009937,-0.026043,-0.029595,-0.007960,-0.039237,0.014291,-0.047486,-0.035713,-0.043166,-0.009384
8,-0.009404,0.002853,0.023786,0.016638,0.028169,-0.019485,-0.000461,-0.003116,0.018629,0.016151
9,-0.008047,-0.017314,0.005507,0.006891,0.018566,0.014566,-0.008133,-0.012803,-0.001811,-0.010320


#### 2.2 По построенной выборке делаем оценку Eest вектора математических ожиданий ...

In [448]:
estE = sample.mean()
estE

MDLZ   -0.000129
MSFT   -0.004899
NXPI   -0.005373
PCAR    0.002668
INTC   -0.002851
NVDA   -0.002502
ILMN   -0.004297
DXCM   -0.008131
ROST   -0.002328
LULU   -0.001632
dtype: float64

#### ... и оценку (σesti,j) матрицы ковариаций. 

In [449]:
estCov = sample.cov()
estCov

,MDLZ,MSFT,NXPI,PCAR,INTC,NVDA,ILMN,DXCM,ROST,LULU
MDLZ,0.000070,0.000021,0.000024,0.000041,0.000065,0.000021,0.000068,0.000023,0.000074,0.000044
MSFT,0.000021,0.000195,0.000112,0.000054,0.000053,0.000182,0.000097,0.000090,0.000064,0.000116
NXPI,0.000024,0.000112,0.000675,0.000236,0.000181,0.000555,0.000134,0.000098,0.000162,0.000195
PCAR,0.000041,0.000054,0.000236,0.000262,0.000144,0.000160,0.000002,0.000026,0.000101,0.000047
INTC,0.000065,0.000053,0.000181,0.000144,0.000351,0.000093,0.000191,0.000063,0.000141,0.000001
NVDA,0.000021,0.000182,0.000555,0.000160,0.000093,0.000898,0.000172,0.000071,0.000082,0.000288
ILMN,0.000068,0.000097,0.000134,0.000002,0.000191,0.000172,0.000439,0.000172,0.000066,0.000067
DXCM,0.000023,0.000090,0.000098,0.000026,0.000063,0.000071,0.000172,0.000321,0.000057,0.000063
ROST,0.000074,0.000064,0.000162,0.000101,0.000141,0.000082,0.000066,0.000057,0.000344,0.000058
LULU,0.000044,0.000116,0.000195,0.000047,0.000001,0.000288,0.000067,0.000063,0.000058,0.000330


In [450]:
import plotly.express as px

fig = px.imshow(cov_matrix, title='Covariation Matrix',
                color_continuous_scale=px.colors.diverging.RdYlGn[::-1])
fig.show()